In [3]:
#FEATURE SELECTION #Chọn các biến
import pandas as pd
df = pd.read_csv('BankChurners.csv')
df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.99994
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.99998
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.99987
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.99998


In [4]:
selected_feature = ['Attrition_Flag',
                   'Gender', 'Dependent_count', 'Education_Level', 'Marital_Status', 'Income_Category', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon']

df = df[selected_feature]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Attrition_Flag            10127 non-null  object
 1   Gender                    10127 non-null  object
 2   Dependent_count           10127 non-null  int64 
 3   Education_Level           10127 non-null  object
 4   Marital_Status            10127 non-null  object
 5   Income_Category           10127 non-null  object
 6   Total_Relationship_Count  10127 non-null  int64 
 7   Months_Inactive_12_mon    10127 non-null  int64 
 8   Contacts_Count_12_mon     10127 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 712.2+ KB


In [5]:
iv_list = []

# loop
for var in selected_feature[1:]:
    new_var = var + '_woe'
    # create woe table
    tmp = df.groupby(var)['Attrition_Flag'].value_counts()
    tmp = pd.DataFrame(tmp)
    tmp = tmp.rename(columns={'Attrition_Flag':'Frequency_of_Loans'})
    tmp = tmp.reset_index()
    woe_table = tmp.pivot(index=var, columns='Attrition_Flag', values='Frequency_of_Loans')

# calculate WOE&IV
num_attrited = woe_table['Attrited Customer'].sum()
num_existing = woe_table['Existing Customer'].sum()

woe_table['DB'] = woe_table['Attrited Customer']/num_attrited
woe_table['DG'] = woe_table['Existing Customer']/num_existing
woe_table['perc_diff'] = woe_table['DG']-woe_table['DB']
woe_table['woe'] = np.log(woe_table['DG']/woe_table['DB'])

# store IV value
iv_data = {'var':var,
               'iv':(woe_table['perc_diff']*woe_table['woe']).sum()}
iv_list.append(iv_data)

# transform categorical into woe
df[new_var] = pd.merge(df, woe_table, how='left', left_on=var,
                                right_index=True)['woe']

KeyError: 'Frequency_of_Loans'

In [6]:
df.head()

,Attrition_Flag,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon
0,Existing Customer,M,3,High School,Married,$60K - $80K,5,1,3
1,Existing Customer,F,5,Graduate,Single,Less than $40K,6,1,2
2,Existing Customer,M,3,Graduate,Married,$80K - $120K,4,1,0
3,Existing Customer,F,4,High School,Unknown,Less than $40K,3,4,1
4,Existing Customer,M,3,Uneducated,Married,$60K - $80K,5,1,0


In [ ]:
# IV
import pandas as pd
iv_table = pd.DataFrame(iv_list) #Tạo khung dữ liệu cho list
#Tạo bảng xong sắp xếp thứ tự các biến tăng dần như trong dataset mẫu
iv_table = iv_table.sort_values('iv', ascending=True).reset_index(drop=True)
#Chọn ra những giá trị thuộc cột iv có unique, đồng thời đổi dạng dữ liệu từ series thành list
iv_table['var'] = pd.Categorical(iv_table['var'], categories=iv_table['var'].unique().tolist(),
                                 ordered=True)
#Trích các loại dữ liệu các kiểu category thuộc cột iv và giá trị thông tin của chúng và in table
iv_table

In [ ]:
iv_table = iv_table.rename(columns={'iv':'information value'})
iv_table.head()
#Thay đổi tên cột trong bảng iv_table

In [ ]:
 #Tạo biểu đồ về mức information value và giá trị trung bình của information value với ggplot
ggplot(iv_table) +\
    geom_col(aes(x='var', y='information value'), fill='black') +\
    geom_hline(yintercept=0.3, linetype='dashed', color='red', size=1.0) +\
    theme(axis_title_y = element_blank()) +\
    coord_flip()

In [ ]:
#CHURNER PROFILE #Tạo profile
#Chọn những giá trị có information value >= 0.3 trong iv_table
selected_features = iv_table[iv_table['information value'] >= 0.3]['var'].unique().tolist()[::-1]
selected_features

In [ ]:
#Đọc file và in các var của các biến
org_df = pd.read_csv('/kaggle/input/credit-card-customers/BankChurners.csv')
org_df.head()

var = ['Attrition_Flag',
       'Total_Trans_Ct',
       'Total_Trans_Amt',
       'Total_Revolving_Bal',
       'Total_Ct_Chng_Q4_Q1',
       'Avg_Utilization_Ratio',
       'Months_Inactive_12_mon']

org_df[var]

In [ ]:
#Tạo biểu đồ var tương quan giữa biến Attrition_Flag và các biến khác
tmp = pd.melt(org_df[var], id_vars='Attrition_Flag')

ggplot(tmp, aes(x='Attrition_Flag', y='value', fill='Attrition_Flag')) +\
    geom_boxplot() +\
    facet_wrap('variable', scales='free_y') +\
    theme(subplots_adjust={'wspace':0.40, 'hspace':0.25},
          axis_text_x=element_blank(),
         axis_title_x=element_blank(),
         axis_title_y=element_blank(),
         figure_size=(16,8))

In [ ]:
#Nhóm dữ liệu theo cột tiêu chí var của Attrition Flag
#Cụ thể các giá trị mean và std giữa Attrition Flag và các biến khác
org_df.groupby('Attrition_Flag')[var].agg([np.mean, np.std])